In [8]:
# Crash Report 2013 data from Professor Miller

import pandas as pd
import numpy as np

#each element in crashes is a list of each category in the crash report
#cant automatically pull the delimited file into a dataframe because they have all have different number of categories
# (bc different number of people/vehicles in the different accidents)
crashes = []

with open("fatals2013.txt") as f:
    #content = f.readlines()
    for line in f:
        b = line.split("|")  # split line into a list of each category
        c = b[0:7]  
        crashes.append(c)

crashdf = pd.DataFrame(crashes)

# we'll want to separate data into different DBs
crashdf.columns = ['crashID','StateTrooper','ID2','date','time','county','location']

crashdf = crashdf.drop(['StateTrooper','ID2'], axis=1)


crashdf.head()
# you may also want to remove whitespace characters like `\n` at the end of each line
#content = [x.strip() for x in content] 




#testline[0]

,crashID,date,time,county,location
0,130000325,01/01/2013,1:30AM,DOUGLAS,"HIGHWAY N, 3 MILES SOUTH OF SQUIRES"
1,130002611,01/02/2013,3:45PM,LEWIS,3RD AND CLARK STREET IN CANTON
2,130005274,01/03/2013,11:35PM,JASPER,ROUTE E AT CR 118 - 1 MILE EAST OF CARTHAGE
3,130009936,01/05/2013,11:25PM,ST. LOUIS,WESTBOUND I-270 EAST OF ROUTE N
4,130010983,01/06/2013,2:15PM,PIKE,US 54 AT ROUTE U


In [53]:
# Make a dictionary of corresponding county names and codes:
countycodes = pd.read_csv('countycodes.csv',header=None, names={'code','county'}, converters={'code': str, 'county': str})

#use 'converter' to keep the county codes as strings and not drop leading zeros
countycodes.head()

crashdf['county'] = crashdf['county'].str.strip()  #get rid of whitespace

#ccdict = countycodes.set_index('county').to_dict()  #make it into a dictionary so we can lookup county names
#^this was doing something weird; see solution below

ccdict = dict(zip(countycodes.code, countycodes.county))

#print(ccdict['ST. FRANCOIS']) #returns '187'

crashdf['ccode'] = crashdf['county'].map(lambda x: ccdict[x], na_action='ignore')

crashdf['fips'] = crashdf['ccode'].apply(lambda x: "29" + str(x))  #in acc_aux we have entire fips codes not just county codes

crashdf.head()

,crashID,date,time,county,ccode,fips
0,130000325,01/01/2013,1:30AM,DOUGLAS,067,29067
1,130002611,01/02/2013,3:45PM,LEWIS,111,29111
2,130005274,01/03/2013,11:35PM,JASPER,097,29097
3,130009936,01/05/2013,11:25PM,ST. LOUIS,189,29189
4,130010983,01/06/2013,2:15PM,PIKE,163,29163


In [50]:
import dbconnect

conn = dbconnect.connect('mgibson_dui')


Connected!



In [54]:
crashdf.to_sql('crash_reports', conn, if_exists='replace')